In [2]:
import pandas as pd
from aerobot.io import load_training_data, load_validation_data, asset_path
from aerobot.lib import process_data
from aerobot.models import LogisticClassifier

print("Loading data...")
feature_type = "aa_1mer"
training_data = load_training_data(feature_type=feature_type)
validation_data = load_validation_data(feature_type=feature_type)
cleaned_data = process_data(training_data["features"], training_data["labels"]["physiology"], validation_data["features"], validation_data["labels"]["physiology"])

Loading data...


In [3]:
cleaned_data

{'X_train': array([[3.20390e+04, 3.59910e+04, 4.17920e+04, ..., 7.65900e+03,
         4.98600e+03, 1.00000e+00],
        [6.36400e+03, 8.07900e+03, 8.27200e+03, ..., 2.09600e+03,
         1.29400e+03, 0.00000e+00],
        [1.33390e+04, 9.86900e+03, 9.36500e+03, ..., 4.66500e+03,
         2.30400e+03, 3.00000e+00],
        ...,
        [1.31050e+05, 3.05630e+05, 1.95761e+05, ..., 5.63920e+04,
         2.33450e+04, 0.00000e+00],
        [3.36790e+04, 4.42390e+04, 3.90500e+04, ..., 1.11000e+04,
         3.11500e+03, 0.00000e+00],
        [6.13830e+04, 1.53054e+05, 1.05113e+05, ..., 3.03910e+04,
         1.36460e+04, 0.00000e+00]]),
 'y_train': array(['Anaerobe', 'Anaerobe', 'Aerobe', ..., 'Aerobe', 'Anaerobe',
        'Aerobe'], dtype=object),
 'X_test': array([[5.78200e+04, 1.09692e+05, 8.59610e+04, ..., 1.75570e+04,
         6.18100e+03, 2.70000e+01],
        [4.41140e+04, 7.58630e+04, 5.97280e+04, ..., 1.86400e+04,
         7.34200e+03, 0.00000e+00],
        [2.33880e+04, 3.48370e+04,

In [6]:
print("Fitting model...")
model = LogisticClassifier(max_iter=100000)
model.fit(cleaned_data["X_train"], cleaned_data["y_train"])
model.score(cleaned_data["X_train"], cleaned_data["y_train"])

Fitting model...


0.7952238805970149

In [7]:
# take test data and fill nans with zeros
X_test = cleaned_data["X_test"]
X_test = pd.DataFrame(X_test, index=cleaned_data["test_row_labels"], columns=cleaned_data["column_labels"])
X_test = X_test.fillna(0).values
model.score(X_test, cleaned_data["y_test"])

0.625

In [22]:
C = model.confusion_matrix(cleaned_data["X_train"], cleaned_data["y_train"])
C= pd.DataFrame(C,index=model.classifier.classes_,columns=model.classifier.classes_)

In [23]:
C.apply(lambda x: x/x.sum(),axis=1)

,Aerobe,Anaerobe,Facultative
Aerobe,0.903188,0.069565,0.027246
Anaerobe,0.116473,0.879368,0.004160
Facultative,0.671395,0.212766,0.115839


In [19]:
C.sum(axis=1)

Aerobe         1739
Anaerobe       1221
Facultative     444
dtype: int64

In [15]:
z = pd.DataFrame(cleaned_data["y_train"])
z["c"] = 1
z.groupby(0).count()

,c
0,
Aerobe,1739
Anaerobe,1221
Facultative,444
